# Cats VS. Dogs
## ECSE 415 - Final Project

### Felix Remigereau and Sébastien Gaspoz

In [3]:
import cv2
import csv
import numpy as np
import matplotlib.pyplot as plt
from sklearn import cross_validation
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

import time

In [4]:
class csvToCv2: 
    def __init__(self, filepath): 
        data = [] 
        labels = []
        
        self.sift = cv2.xfeatures2d.SIFT_create()
        self.descs = []
        
        with open(filepath+'Y_train.csv', 'r') as csv_file:
            # read and discard the csv header row (first line)
            next(csv_file)
            # Read the rest, one line at a time
            for line in csv.reader(csv_file):
                # The first column contains the image's name
                im = cv2.imread(filepath+'X_train/'+str(line[0]))
                im = cv2.resize(im, (320, 240))
                im.reshape(-1)
                data.append(im)
                # The last column is the label                
                labels.append(int(line[1]))
                # The rest of columns are pixels
                
        self.data = data
        self.labels = labels
    
    def fasterFeatures(self): 
        feat = []
        desc = []
        for pic in self.data[:100]: 
            feature, descriptor = self.sift.detectAndCompute(pic, None)
            feat.append(feature)
            desc.append(descriptor)
            
        self.feats = feat
        self.descs = desc
    
    
    def computeFeatures(self) :
        #feat = []
        desc = []
        k = 0
        for pic in self.data[:100]: 
            if(k % 20 == 0):
                print("Doing stuff for picture : ", k)
            k += 1
            feature, descriptor = self.sift.detectAndCompute(pic, None)
            
            zi = zip(feature, descriptor) # We want to be able to sort the descriptors based on the keypoints attribute, so we zip them
            bestFeats = sorted(zi, key=lambda tup : tup[0].response, reverse = True)[:100]
            for pt in bestFeats:
                #feat.append(pt[0])
                desc.append(pt[1])
            #self.feats.append(feat)
            self.descs.append(desc)
        
       
    def splitData(self):
        # Spliting the training data into training and validation sets
        # We'll use the cross_validation from the sklearn module
        Xtrain,Xval,ytrain,yval = cross_validation.train_test_split(self.descs,self.labels,test_size=0.1)
        # make the data public
        self.Xtrain = Xtrain
        self.ytrain = ytrain
        self.Ntrain = len(Xtrain)
        self.Xval = Xval
        self.yval = yval
        self.Nval = len(Xval)
        
    # Displays N images for each animal from the training set
    def demo(self,N):
        # keeps track of the number of plotted images for each digit
        nplot = np.zeros(2)
        # Go through all images
        for n in range(self.Ntrain):        
            # get the digit label
            digit = self.ytrain[n]
            # Check the number of plotted images for the digit
            if nplot[digit] < N:
                nplot[digit] = nplot[digit] + 1
                # read and reshape the digit image
                img = self.Xtrain[n]
                # compute the subplot id
                plotID = N * digit + nplot[digit]
                # plot
                plt.subplot(2,N,plotID),plt.imshow(img, cmap='gray')
                plt.xticks([]), plt.yticks([])
                
            # break if done
            if nplot.min() == N:
                break
        # Display
        plt.show()        

In [5]:
print("hey")
start = time.clock()
picturesDirectory = "C:/Users/yanng/Documents/Vision/Projet/"
test = csvToCv2(picturesDirectory)
test.fasterFeatures()
#test.splitData()
finish = time.clock()
print(finish - start)

hey
222.52175109603508


In [ ]:
# Create an sklearn.svm.LinearSVC object
svm = LinearSVC()
# Train using the training set
svm.fit(test.Xtrain, test.ytrain)
# Predict the labels for the validation set
ypred = svm.predict(test.Xval)
# Compute the prediction accuracy (%) using the accuracy_score module of the sklearn.metrics
accuracy = accuracy_score(test.yval,ypred) * 100
print("SVM accuracy: ", accuracy) 

In [24]:
# Create an sklearn.ensemble.RandomForestClassifier object
randomforest = RandomForestClassifier(n_estimators=10, n_jobs=-1)
# Train using the training set
randomforest.fit(test.Xtrain, test.ytrain)
# Predict the labels for the validation set
ypred = randomforest.predict(test.Xval)
# Compute the prediction accuracy (%) using the accuracy_score module of the sklearn.metrics
accuracy = accuracy_score(test.yval,ypred) * 100
print("random forest accuracy: ", accuracy) 

MemoryError: 

In [ ]:
print(len(test.desc)/100) 

In [14]:
test.descs

[]

In [19]:
cv2.imshow("Datar", test.data[0])